In [1]:
# Importing the required library for handling HDF5 files
import h5py

# Opening the HDF5 file
file_path = '/home/ubuntu/data/de_111496386_20211021101732_20211021102136.hd5'
hdf5_file = h5py.File(file_path, 'r')

# Function to explore the structure of the HDF5 file recursively
def explore_hdf5_group(group, indent=0):
    items = []
    for key, value in group.items():
        if isinstance(value, h5py.Group):
            items.append(("  " * indent + f"Group: {key}", explore_hdf5_group(value, indent + 1)))
        elif isinstance(value, h5py.Dataset):
            items.append({key})
            #print({key})
        else:
            items.append(("  " * indent + f"Unknown type: {key}", str(type(value))))
            print({key})
    return items

# Exploring the root of the HDF5 file
structure = explore_hdf5_group(hdf5_file)
print("structure:",structure)

# Function to extract metadata from the HDF5 group
def extract_metadata(group):
    metadata = {}
    for key, value in group.items():
        # Exclude the datasets that represent ECG leads
        #if key not in ["I", "II", "III", "V1", "V2", "V3", "V4", "V5", "V6", "aVF", "aVL", "aVR"]:
        if value.shape == ():  # Check if the dataset is a scalar value
            metadata[key] = value[()]
            print(key,metadata[key])
        else:
            metadata[key] = value[:]  # For non-scalar values, get the entire array
            #print(key,metadata[key])
    return metadata
'''
# Extracting metadata from the specific group
t = hdf5_file['ecg'].keys()
print(t)
ecg_group = hdf5_file['ecg'][t]
metadata = extract_metadata(ecg_group)
print("metadata:",metadata)
'''


for k in list(hdf5_file['ecg'].keys()):
    t = hdf5_file['ecg'][k]
    ecg = []
    metadata = extract_metadata(t)




structure: [('Group: ecg', [('  Group: 2021-10-21T00:00:00', [{'I'}, {'II'}, {'III'}, {'V1'}, {'V2'}, {'V3'}, {'V4'}, {'V5'}, {'V6'}, {'aVF'}, {'aVL'}, {'aVR'}, {'admitdate'}, {'admittime'}, {'atrialrate_md'}, {'atrialrate_pc'}, {'cartnumber'}, {'datatype'}, {'dateofbirth'}, {'ecgsamplebase_md'}, {'ecgsamplebase_pc'}, {'ecgsampleexponent_md'}, {'ecgsampleexponent_pc'}, {'editdate'}, {'editliststatus'}, {'edittime'}, {'gender'}, {'hisdisposition'}, {'hisstatus'}, {'orderplacementdate'}, {'orderplacementtime'}, {'ordertime'}, {'patientid'}, {'patientid_clean'}, {'paxis_md'}, {'paxis_pc'}, {'poffset_md'}, {'poffset_pc'}, {'ponset_md'}, {'ponset_pc'}, {'printerval_md'}, {'printerval_pc'}, {'priority'}, {'qoffset_md'}, {'qoffset_pc'}, {'qonset_md'}, {'qonset_pc'}, {'qrscount_md'}, {'qrscount_pc'}, {'qrsduration_md'}, {'qrsduration_pc'}, {'qtcfrederica_md'}, {'qtcfrederica_pc'}, {'qtcorrected_md'}, {'qtcorrected_pc'}, {'qtinterval_md'}, {'qtinterval_pc'}, {'raxis_md'}, {'raxis_pc'}, {'referr

In [5]:
import os
import pandas as pd

def process_and_combine_excel_files(directory_path):
    # List all files in the directory
    all_files = os.listdir(directory_path)
    excel_files = [f for f in all_files if f.endswith('.xlsx') or f.endswith('.xls')]

    # Read and process each excel file
    dataframes = []
    for file in excel_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_excel(file_path)
        # Replace "|" with a space
        df['deid_t_diagnosis_original'] = df['deid_t_diagnosis_original'].str.replace('|', ' ')
        dataframes.append(df)

    # Combine all dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Save combined dataframe to a CSV
    combined_csv_path = os.path.join(directory_path, 'combined_ecg_diagnosis.csv')
    combined_df.to_csv(combined_csv_path, index=False)
    
    return combined_csv_path

combined_file_path = "/home/ubuntu/data/"
process_and_combine_excel_files(combined_file_path)

'/home/ubuntu/data/combined_ecg_diagnosis.csv'

In [2]:
import os
import pandas as pd
map_label = pd.read_csv('/home/ubuntu/data/map_label.csv')
sample = pd.read_csv('/home/ubuntu/data/combined_ecg_diagnosis.csv')

def one_hot_encoding(row, labels):
    """Return one-hot encoding for a given row based on the provided labels."""
    if pd.isna(row):
        return [0] * len(labels)
    return [int(label in row) for label in labels]

# Apply one-hot encoding for each row in sample['deid_t_diagnosis']
sample['label'] = sample['deid_t_diagnosis'].apply(lambda x: one_hot_encoding(x, map_label['Text']))
sample.to_csv('/home/ubuntu/data/ecg/ecg_label.csv', index=False)
# Display the first few rows of the modified sample DataFrame
sample.head()

,Unnamed: 0,bdsp_id,HashFileName,deid_t_diagnosis,deid_t_diagnosis_original,label
0,0,111888311,de_111888311_20120607205223_20120608195341.hd5,NORMAL SINUS RHYTHM|NORMAL ECG|WHEN COMPARED W...,NORMAL SINUS RHYTHM NORMAL ECG WHEN COMPARED W...,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,114308535,de_114308535_20120701090717_20120703143853.hd5,SINUS RHYTHM|WITH FREQUENT|PREMATURE VENTRICUL...,SINUS RHYTHM WITH FREQUENT PREMATURE VENTRICUL...,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,2,115828429,de_115828429_20111219012630_20111219114122.hd5,NORMAL SINUS RHYTHM|NORMAL ECG|WHEN COMPARED W...,NORMAL SINUS RHYTHM NORMAL ECG WHEN COMPARED W...,"[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,120680337,de_120680337_20110601084937_20110601204411.hd5,NORMAL SINUS RHYTHM|NONSPECIFIC T WAVE ABNORMA...,NORMAL SINUS RHYTHM NONSPECIFIC T WAVE ABNORMA...,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,4,119019508,de_119019508_20120214153908_20120215103635.hd5,NORMAL SINUS RHYTHM|LEFT AXIS DEVIATION|ABNORM...,NORMAL SINUS RHYTHM LEFT AXIS DEVIATION ABNORM...,"[1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [7]:
sample['label'].head(20)

0     [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
2     [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
4     [1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
5     [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6     [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
7     [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8     [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9     [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11    [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
12    [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
13    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
14    [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
15    [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
16    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
17    [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,